In [1]:
import pandas as pd
import s3fs
import boto3
import json
from datetime import datetime
import numpy as np

In [14]:
SAGEMAKER_REPO_PATH = r'/home/ec2-user/SageMaker/classify-streetview'

In [2]:
df_results = pd.read_csv('all_test_crops_model0.csv')
df_results.shape

(3448, 8)

In [3]:
df_results = df_results.rename(columns = {'Unnamed: 0.1' : 'image_name'})
df_results.head()


,Unnamed: 0,image_name,prediction,0,1,2,3,4
0,0,8475_135_1.jpg,1_null,-1.133551,4.186599,-1.984925,1.529789,-2.481364
1,1,8475_135_2.jpg,1_null,-0.763638,3.575558,-1.084529,1.076634,-2.774253
2,2,8475_135_3.jpg,1_null,-1.193419,4.510900,-2.012558,0.620944,-1.824119
3,3,8475_135_4.jpg,1_null,-0.106576,3.914730,-2.002850,1.609854,-3.344971
4,4,8475_135_5.jpg,1_null,-1.827574,4.439346,-1.939240,0.628309,-1.315913


In [4]:
df_results.columns

Index(['Unnamed: 0', 'image_name', 'prediction', '0', '1', '2', '3', '4'], dtype='object')

In [5]:
df_results = df_results[['image_name', 'prediction', '0', '1', '2', '3', '4']]

In [7]:
df_test = pd.read_csv(os.path.join(SAGEMAKER_REPO_PATH, 'label-crops/test_crop_filename.csv'))
df_test.head()

,Unnamed: 0,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,img_heading_folder,original_image_path,new_image_path,image_name
0,0,141,45,1,0,1,0,0,0,0,0,0,141_45,crops/141_45/1.jpg,labeled-test/141_45_1.jpg,141_45_1.jpg
1,1,141,45,2,0,1,0,0,0,0,0,0,141_45,crops/141_45/2.jpg,labeled-test/141_45_2.jpg,141_45_2.jpg
2,2,141,45,3,0,1,0,0,0,0,0,0,141_45,crops/141_45/3.jpg,labeled-test/141_45_3.jpg,141_45_3.jpg
3,3,141,45,4,0,1,0,0,0,0,0,0,141_45,crops/141_45/4.jpg,labeled-test/141_45_4.jpg,141_45_4.jpg
4,4,141,45,5,0,1,0,0,0,0,0,0,141_45,crops/141_45/5.jpg,labeled-test/141_45_5.jpg,141_45_5.jpg


In [8]:
df_test = df_test[['img_id', 'heading', 'crop_number', '0_missing', '1_null',
       '2_obstacle', '3_present', '4_surface_prob', '5_nosidewalk',
       '6_occlusion', 'count_all', 'image_name']]

In [9]:
df_combine = df_results.merge(df_test, how = 'left', left_on = 'image_name', right_on = 'image_name')
df_combine.head()

,image_name,prediction,0,1,2,3,4,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all
0,8475_135_1.jpg,1_null,-1.133551,4.186599,-1.984925,1.529789,-2.481364,8475,135,1,0,1,0,0,0,0,0,0
1,8475_135_2.jpg,1_null,-0.763638,3.575558,-1.084529,1.076634,-2.774253,8475,135,2,0,0,1,0,0,0,0,1
2,8475_135_3.jpg,1_null,-1.193419,4.510900,-2.012558,0.620944,-1.824119,8475,135,3,0,1,0,0,0,0,0,0
3,8475_135_4.jpg,1_null,-0.106576,3.914730,-2.002850,1.609854,-3.344971,8475,135,4,0,1,0,0,0,0,0,0
4,8475_135_5.jpg,1_null,-1.827574,4.439346,-1.939240,0.628309,-1.315913,8475,135,5,0,0,1,0,0,0,0,1


In [10]:
df_combine['ground_truth'] = '1_null'
df_combine['ground_truth'] = np.where(df_combine['3_present'] > 0, '3_present', df_combine['ground_truth'])
df_combine['ground_truth'] = np.where(df_combine['0_missing'] > 0, '0_missing', df_combine['ground_truth'])
df_combine['ground_truth'] = np.where(df_combine['4_surface_prob'] > 0, '4_surface_prob', df_combine['ground_truth'])
df_combine['ground_truth'] = np.where(df_combine['2_obstacle'] > 0, '2_obstacle', df_combine['ground_truth'])
df_combine['ground_truth'].value_counts()

1_null            2268
3_present          718
4_surface_prob     250
2_obstacle         129
0_missing           83
Name: ground_truth, dtype: int64

In [11]:
df_combine['prediction'].value_counts()

1_null        3292
3_present      149
2_obstacle       7
Name: prediction, dtype: int64

In [12]:

df_combine.head()

,image_name,prediction,0,1,2,3,4,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,ground_truth
0,8475_135_1.jpg,1_null,-1.133551,4.186599,-1.984925,1.529789,-2.481364,8475,135,1,0,1,0,0,0,0,0,0,1_null
1,8475_135_2.jpg,1_null,-0.763638,3.575558,-1.084529,1.076634,-2.774253,8475,135,2,0,0,1,0,0,0,0,1,2_obstacle
2,8475_135_3.jpg,1_null,-1.193419,4.510900,-2.012558,0.620944,-1.824119,8475,135,3,0,1,0,0,0,0,0,0,1_null
3,8475_135_4.jpg,1_null,-0.106576,3.914730,-2.002850,1.609854,-3.344971,8475,135,4,0,1,0,0,0,0,0,0,1_null
4,8475_135_5.jpg,1_null,-1.827574,4.439346,-1.939240,0.628309,-1.315913,8475,135,5,0,0,1,0,0,0,0,1,2_obstacle


In [13]:
df_combine.to_csv('model0predict_with_groundtruth.csv')